In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import missingno

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [10]:
data = pd.read_csv('data\premier league\england-premier-league-matches-2023-to-2024-stats.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 67 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   timestamp                            380 non-null    int64  
 1   date_GMT                             380 non-null    object 
 2   status                               380 non-null    object 
 3   attendance                           66 non-null     float64
 4   home_team_name                       380 non-null    object 
 5   away_team_name                       380 non-null    object 
 6   referee                              379 non-null    object 
 7   Game Week                            380 non-null    int64  
 8   Pre-Match PPG (Home)                 380 non-null    float64
 9   Pre-Match PPG (Away)                 380 non-null    float64
 10  home_ppg                             380 non-null    float64
 11  away_ppg                        

In [5]:
data.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league
0,1691780400,Aug 11 2023 - 7:00pm,complete,21572.0,Burnley,Manchester City,Craig Pawson,1,0.0,0.0,0.53,2.32,0,3,3,2,0,2,NaN,"4,36,75",6,5,0,1,0,0,0,1,0,0,5,13,2,9,3,4,11,8,34,66,0.0,0.0,0.65,1.72,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,11.00,6.30,1.28,1.20,1.56,2.65,4.85,2.05,1.70,Turf Moor (Burnley),premier league
1,1691841600,Aug 12 2023 - 12:00pm,complete,59984.0,Arsenal,Nottingham Forest,Michael Oliver,1,0.0,0.0,2.47,0.84,2,1,3,2,2,0,"26,32",82,8,3,2,0,2,0,1,1,1,1,11,6,8,3,3,3,12,12,78,22,0.0,0.0,1.72,0.76,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.25,6.25,9.50,1.11,1.45,2.01,3.45,2.00,1.75,Emirates Stadium (London),premier league
2,1691848800,Aug 12 2023 - 2:00pm,complete,NaN,AFC Bournemouth,West Ham United,Peter Bankes,1,0.0,0.0,1.42,1.21,1,1,2,0,0,0,82,51,10,4,1,0,4,0,0,1,1,3,13,13,6,4,7,9,8,14,63,37,0.0,0.0,1.66,1.39,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.63,3.06,2.35,1.29,1.89,3.00,5.75,1.70,2.05,Vitality Stadium (Bournemouth- Dorset),premier league
3,1691848800,Aug 12 2023 - 2:00pm,complete,NaN,Brighton & Hove Albion,Luton Town,David Coote,1,0.0,0.0,1.58,0.53,4,1,5,1,1,0,"36,71,85,90'5",81,6,7,2,0,2,0,2,0,0,2,23,8,13,4,10,4,4,8,71,29,0.0,0.0,2.91,1.06,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.34,4.70,7.00,1.18,1.58,2.38,4.90,2.00,1.75,The American Express Community Stadium (Falmer...,premier league
4,1691848800,Aug 12 2023 - 2:00pm,complete,NaN,Everton,Fulham,Stuart Attwell,1,0.0,0.0,1.47,0.95,0,1,1,0,0,0,NaN,73,10,4,0,0,2,0,0,0,1,1,16,9,10,3,6,6,12,6,41,59,0.0,0.0,2.09,1.05,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.32,3.10,2.89,1.29,2.08,3.40,7.00,1.80,1.95,Goodison Park (Liverpool),premier league


In [11]:
data = clean_data(data)

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_2548\1461863474.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date_GMT'] = pd.to_datetime(data['date_GMT'])


In [33]:
data_folder = './data'

for entry in os.listdir(data_folder):
    entry_path = os.path.join(data_folder, entry)
    if os.path.isdir(entry_path):
        for file_name in os.listdir(entry_path):
            if file_name.endswith('.csv'):
                file_path = os.path.join(entry_path, file_name)
                df = pd.read_csv(file_path)
                df['league'] = entry
                df.to_csv(file_path, index=False)

In [34]:
data = pd.DataFrame()

for folder in os.listdir('./data'):
    if folder.endswith('csv'):
        continue
    for file in os.listdir(f'./data/{folder}'):
        df = pd.read_csv(f'./data/{folder}/{file}')
        data = pd.concat([data, df], ignore_index=True)

In [35]:
data.isna().sum()

timestamp                                  0
date_GMT                                   0
status                                     0
attendance                             30917
home_team_name                             0
away_team_name                             0
referee                                31947
Game Week                               6853
Pre-Match PPG (Home)                       0
Pre-Match PPG (Away)                       0
home_ppg                                   0
away_ppg                                   0
home_team_goal_count                       0
away_team_goal_count                       0
total_goal_count                           0
total_goals_at_half_time                   0
home_team_goal_count_half_time             0
away_team_goal_count_half_time             0
home_team_goal_timings                 19417
away_team_goal_timings                 27086
home_team_corner_count                     0
away_team_corner_count                     0
home_team_

In [7]:
def clean_data(data):
    #transforming date variable
    data['date_GMT'] = pd.to_datetime(data['date_GMT'])
    data['date'] = pd.to_datetime(data['date_GMT'].dt.date)
    data['time'] = data['date_GMT'].dt.time
    
    #adding aggregated variables
    data['corners_total'] = data['home_team_corner_count'] + data['away_team_corner_count']
    data['yellow_cards_total'] = data['home_team_yellow_cards'] + data['away_team_yellow_cards']
    data['red_cards_total'] = data['home_team_red_cards'] + data['away_team_red_cards']
    data['cards_total'] = data['yellow_cards_total'] + data['red_cards_total']
    data['shots_total'] = data['home_team_shots'] + data['away_team_shots']
    data['shots_on_target_total'] = data['home_team_shots_on_target'] + data['away_team_shots_on_target']
    data['shots_off_target_total'] = data['home_team_shots_off_target'] + data['away_team_shots_off_target']
    data['fouls_total'] = data['home_team_fouls'] + data['away_team_fouls']
    
    #getting stadium names without city in brackets
    # data['base_name'] = data['stadium_name'].str.replace(r" \(.*\)$", "", regex=True)

    # city_map = data[data['stadium_name'].str.contains(r"\(.*\)")].copy()
    # city_map['city'] = city_map['stadium_name'].str.extract(r"\((.*?)\)")[0]
    # city_map = city_map.groupby('base_name')['city'].agg(pd.Series.mode).to_dict()

    # #filling stadium names without city in brackets
    # data['normalized_stadium'] = data.apply(lambda row: f"{row['base_name']} ({city_map.get(row['base_name'], 'Unknown')})" if '(' not in row['stadium_name'] else row['stadium_name'], axis=1)

    #dropping unnecessary features
    data.drop(['timestamp', 'status', 'home_team_goal_timings', 'away_team_goal_timings', 'date_GMT'], axis=1, inplace=True)
    
    #setting one of the target variable - result of the game
    data['result'] = np.where(data['home_team_goal_count'] == data['away_team_goal_count'], 0, np.where(data['home_team_goal_count'] > data['away_team_goal_count'], 1, 2))
    
    def assign_season(date):
        year = date.year
        if date.month >= 8:  # Sezon zaczyna się w sierpniu
            return f'{str(year)[2:]}/{str(year+1)[2:]}'
        else:
            return f'{str(year-1)[2:]}/{str(year)[2:]}'
    
    data['season'] = data['date'].apply(assign_season)
    
    return data

In [39]:
data = clean_data(data)

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_23700\1461863474.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date_GMT'] = pd.to_datetime(data['date_GMT'])


In [40]:
data.shape

(83142, 74)

In [41]:
data.head()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season
0,8016.0,Austria Wien,Admira,NaN,1.0,0.0,0.0,1.61,0.67,2,0,2,0,0,0,6,7,2,0,5,0,0,2,1,4,13,13,7,6,6,7,9,17,53,47,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.36,5.15,9.65,0.0,0.0,0.0,0.0,0.0,0.0,Generali Arena (Wien),austria,2013-07-20,14:30:00,13,7,0,7,26,13,13,26,1,12/13
1,2840.0,Grödig,Ried,NaN,1.0,0.0,0.0,1.44,0.94,0,0,0,0,0,0,6,11,8,0,5,0,5,3,2,3,8,16,4,10,4,6,20,24,49,51,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.70,3.42,2.73,0.0,0.0,0.0,0.0,0.0,0.0,DAS.GOLDBERG Stadion,austria,2013-07-20,17:00:00,17,13,0,13,24,14,10,44,0,12/13
2,2696.0,Wiener Neustadt,Salzburg,NaN,1.0,0.0,0.0,1.11,1.94,1,5,6,3,1,2,3,10,4,0,1,0,0,4,0,1,5,18,2,6,3,12,13,16,52,48,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,6.97,4.51,1.51,0.0,0.0,0.0,0.0,0.0,0.0,Teddybären&Plüsch-Stadion,austria,2013-07-20,17:00:00,13,5,0,5,23,8,15,29,2,12/13
3,6413.0,Wolfsberger AC,Rapid Wien,NaN,1.0,0.0,0.0,1.33,1.33,2,2,4,2,0,2,6,3,2,0,4,0,0,2,2,2,7,13,3,6,4,7,8,26,51,49,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,3.29,3.37,2.33,0.0,0.0,0.0,0.0,0.0,0.0,Lavanttal Arena,austria,2013-07-20,17:00:00,9,6,0,6,20,9,11,34,0,12/13
4,7406.0,Wacker Innsbruck,Sturm Graz,NaN,1.0,0.0,0.0,0.94,1.44,2,2,4,0,0,0,7,1,2,0,4,0,0,2,0,4,15,5,5,3,10,2,21,20,48,52,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.54,3.37,2.93,0.0,0.0,0.0,0.0,0.0,0.0,Tivoli Stadion Tirol,austria,2013-07-21,14:30:00,8,6,0,6,20,8,12,41,0,12/13


In [42]:
data.isna().sum()

attendance                             30917
home_team_name                             0
away_team_name                             0
referee                                31947
Game Week                               6853
Pre-Match PPG (Home)                       0
Pre-Match PPG (Away)                       0
home_ppg                                   0
away_ppg                                   0
home_team_goal_count                       0
away_team_goal_count                       0
total_goal_count                           0
total_goals_at_half_time                   0
home_team_goal_count_half_time             0
away_team_goal_count_half_time             0
home_team_corner_count                     0
away_team_corner_count                     0
home_team_yellow_cards                     0
home_team_red_cards                        0
away_team_yellow_cards                     0
away_team_red_cards                        0
home_team_first_half_cards                 0
home_team_

# Attendance

In [13]:
# Pierwsze wypełnienie: średnia dla gospodarza i sezonu
data['attendance'] = data['attendance'].fillna(data.groupby(['home_team_name', 'season'])['attendance'].transform('mean'))

In [15]:
data['attendance'].isna().sum()

16647

In [16]:
# Drugie wypełnienie: średnia dla ligi i sezonu
data['attendance'] = data['attendance'].fillna(data.groupby(['league', 'season'])['attendance'].transform('mean'))

In [18]:
data['attendance'].isna().sum()

8968

In [19]:
# Trzecie wypełnienie: średnia dla ligi na przestzeni całego datasetu
data['attendance'] = data['attendance'].fillna(data.groupby(['league'])['attendance'].transform('mean'))

In [20]:
data['attendance'].isna().sum()

0

# Rolling variables

In [21]:
# Funkcja do obliczania średniej strzałów dla drużyny (zarówno dom jak i wyjazd)
def rolling_shots_avg(data, team_col, shots_col):
    return data.groupby(team_col)[shots_col].transform(lambda x: x.rolling(window=5, min_periods=1).mean())

In [22]:
# Obliczanie średniej liczby strzałów dla drużyn domowych i wyjazdowych
data['home_team_shots_rolling_avg'] = rolling_shots_avg(data, 'home_team_name', 'home_team_shots')
data['away_team_shots_rolling_avg'] = rolling_shots_avg(data, 'away_team_name', 'away_team_shots')

In [24]:
# Tworzenie nowych kolumn dla sumy kartek
data['home_team_total_cards'] = data['home_team_yellow_cards'] + data['home_team_red_cards']
data['away_team_total_cards'] = data['away_team_yellow_cards'] + data['away_team_red_cards']

In [25]:
# Funkcja do obliczania średniej ruchomej dla danych statystyk
def rolling_avg(data, team_col, stat_col):
    return data.groupby(team_col)[stat_col].transform(lambda x: x.rolling(window=5, min_periods=1).mean())

# Obliczanie średnich dla goli, rożnych i kartek
# Dla goli
data['home_team_goal_count_rolling_avg'] = rolling_avg(data, 'home_team_name', 'home_team_goal_count')
data['away_team_goal_count_rolling_avg'] = rolling_avg(data, 'away_team_name', 'away_team_goal_count')

# Dla rzutów rożnych
data['home_team_corner_count_rolling_avg'] = rolling_avg(data, 'home_team_name', 'home_team_corner_count')
data['away_team_corner_count_rolling_avg'] = rolling_avg(data, 'away_team_name', 'away_team_corner_count')

# Dla kartek
data['home_team_cards_rolling_avg'] = rolling_avg(data, 'home_team_name', 'home_team_total_cards')
data['away_team_cards_rolling_avg'] = rolling_avg(data, 'away_team_name', 'away_team_total_cards')

In [26]:
data.head()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,home_team_shots_rolling_avg,away_team_shots_rolling_avg,home_team_total_cards,away_team_total_cards,home_team_goal_count_rolling_avg,away_team_goal_count_rolling_avg,home_team_corner_count_rolling_avg,away_team_corner_count_rolling_avg,home_team_cards_rolling_avg,away_team_cards_rolling_avg
0,8016.0,Austria Wien,Admira,NaN,1.0,0.0,0.0,1.61,0.67,2,0,2,0,0,0,6,7,2,0,5,0,0,2,1,4,13,13,7,6,6,7,9,17,53,47,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.36,5.15,9.65,0.0,0.0,0.0,0.0,0.0,0.0,Generali Arena (Wien),austria,2013-07-20,14:30:00,13,7,0,7,26,13,13,26,1,12/13,13.0,13.0,2,5,2.0,0.0,6.0,7.0,2.0,5.0
1,2840.0,Grödig,Ried,NaN,1.0,0.0,0.0,1.44,0.94,0,0,0,0,0,0,6,11,8,0,5,0,5,3,2,3,8,16,4,10,4,6,20,24,49,51,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.70,3.42,2.73,0.0,0.0,0.0,0.0,0.0,0.0,DAS.GOLDBERG Stadion,austria,2013-07-20,17:00:00,17,13,0,13,24,14,10,44,0,12/13,8.0,16.0,8,5,0.0,0.0,6.0,11.0,8.0,5.0
2,2696.0,Wiener Neustadt,Salzburg,NaN,1.0,0.0,0.0,1.11,1.94,1,5,6,3,1,2,3,10,4,0,1,0,0,4,0,1,5,18,2,6,3,12,13,16,52,48,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,6.97,4.51,1.51,0.0,0.0,0.0,0.0,0.0,0.0,Teddybären&Plüsch-Stadion,austria,2013-07-20,17:00:00,13,5,0,5,23,8,15,29,2,12/13,5.0,18.0,4,1,1.0,5.0,3.0,10.0,4.0,1.0
3,6413.0,Wolfsberger AC,Rapid Wien,NaN,1.0,0.0,0.0,1.33,1.33,2,2,4,2,0,2,6,3,2,0,4,0,0,2,2,2,7,13,3,6,4,7,8,26,51,49,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,3.29,3.37,2.33,0.0,0.0,0.0,0.0,0.0,0.0,Lavanttal Arena,austria,2013-07-20,17:00:00,9,6,0,6,20,9,11,34,0,12/13,7.0,13.0,2,4,2.0,2.0,6.0,3.0,2.0,4.0
4,7406.0,Wacker Innsbruck,Sturm Graz,NaN,1.0,0.0,0.0,0.94,1.44,2,2,4,0,0,0,7,1,2,0,4,0,0,2,0,4,15,5,5,3,10,2,21,20,48,52,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.54,3.37,2.93,0.0,0.0,0.0,0.0,0.0,0.0,Tivoli Stadion Tirol,austria,2013-07-21,14:30:00,8,6,0,6,20,8,12,41,0,12/13,15.0,5.0,2,4,2.0,2.0,7.0,1.0,2.0,4.0


# Game week

In [42]:
df = pd.read_csv('./data/premier league/england-premier-league-matches-2007-to-2008-stats.csv')

In [43]:
df

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league
0,1186832700,Aug 11 2007 - 11:45am,complete,NaN,Sunderland,Tottenham Hotspur,NaN,NaN,0.00,0.00,1.58,0.89,1,0,1,0,0,0,90'3,NaN,-1,-1,1,0,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stadium of Light (Sunderland),premier league
1,1186840800,Aug 11 2007 - 2:00pm,complete,NaN,West Ham United,Manchester City,NaN,NaN,0.00,0.00,1.47,0.95,0,2,2,1,0,1,NaN,"18,87",-1,-1,0,0,4,0,0,0,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Boleyn Ground (London),premier league
2,1186840800,Aug 11 2007 - 2:00pm,complete,NaN,Middlesbrough,Blackburn Rovers,NaN,NaN,0.00,0.00,1.37,1.42,1,2,3,1,1,0,30,"63,79",-1,-1,3,0,4,0,2,1,0,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Riverside Stadium (Middlesbrough),premier league
3,1186840800,Aug 11 2007 - 2:00pm,complete,NaN,Everton,Wigan Athletic,NaN,NaN,0.00,0.00,1.95,0.58,2,1,3,1,1,0,"26,75",80,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Goodison Park (Liverpool),premier league
4,1186840800,Aug 11 2007 - 2:00pm,complete,NaN,Derby County,Portsmouth,NaN,NaN,0.00,0.00,0.42,1.47,2,2,4,2,1,1,"4,84","27,83",-1,-1,1,0,2,0,0,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pride Park Stadium (Derby),premier league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1210514400,May 11 2008 - 2:00pm,complete,NaN,West Ham United,Aston Villa,NaN,NaN,1.50,1.44,1.47,1.42,2,2,4,2,1,1,"8,88","14,58",-1,-1,2,0,2,0,2,0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,2.94,61,86,56,28,9,28,78,48,86,0.0,3.29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Boleyn Ground (London),premier league
376,1210514400,May 11 2008 - 2:00pm,complete,NaN,Tottenham Hotspur,Liverpool,NaN,NaN,1.61,1.72,1.53,1.79,0,2,2,0,0,0,NaN,"69,74",-1,-1,1,0,1,0,0,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,3.20,58,84,58,39,17,28,75,56,86,0.0,2.72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,White Hart Lane (London),premier league
377,1210514400,May 11 2008 - 2:00pm,complete,NaN,Sunderland,Arsenal,NaN,NaN,1.67,1.83,1.58,1.89,0,1,1,1,0,1,NaN,24,-1,-1,1,0,1,0,0,1,0,1,-1,-1,-1,-1,-1,-1,-1,

In [46]:
import pandas as pd

# Funkcja do przetwarzania jednego pliku
def process_season(df):
    df['date_GMT'] = pd.to_datetime(df['date_GMT'], format='%b %d %Y - %I:%M%p')
    df = df.sort_values(by='date_GMT')
    if 'Game Week' not in df.columns or df['Game Week'].isna().any():
        df['Game Week'] = df['date_GMT'].rank(method='dense').astype(int)
    return df

# Przykład użycia funkcji
processed_season_df = process_season(df)
processed_season_df.to_csv('path_to_processed_season_csv_file.csv', index=False)

In [47]:
process_season = pd.read_csv('path_to_processed_season_csv_file.csv')

In [48]:
process_season

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league
0,1186832700,2007-08-11 11:45:00,complete,NaN,Sunderland,Tottenham Hotspur,NaN,1,0.00,0.00,1.58,0.89,1,0,1,0,0,0,90'3,NaN,-1,-1,1,0,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stadium of Light (Sunderland),premier league
1,1186840800,2007-08-11 14:00:00,complete,NaN,West Ham United,Manchester City,NaN,2,0.00,0.00,1.47,0.95,0,2,2,1,0,1,NaN,"18,87",-1,-1,0,0,4,0,0,0,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Boleyn Ground (London),premier league
2,1186840800,2007-08-11 14:00:00,complete,NaN,Middlesbrough,Blackburn Rovers,NaN,2,0.00,0.00,1.37,1.42,1,2,3,1,1,0,30,"63,79",-1,-1,3,0,4,0,2,1,0,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Riverside Stadium (Middlesbrough),premier league
3,1186840800,2007-08-11 14:00:00,complete,NaN,Everton,Wigan Athletic,NaN,2,0.00,0.00,1.95,0.58,2,1,3,1,1,0,"26,75",80,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Goodison Park (Liverpool),premier league
4,1186840800,2007-08-11 14:00:00,complete,NaN,Derby County,Portsmouth,NaN,2,0.00,0.00,0.42,1.47,2,2,4,2,1,1,"4,84","27,83",-1,-1,1,0,2,0,0,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pride Park Stadium (Derby),premier league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1210514400,2008-05-11 14:00:00,complete,NaN,Wigan Athletic,Manchester United,NaN,200,1.61,1.78,1.53,1.84,0,2,2,1,0,1,NaN,"33,80",-1,-1,3,0,2,0,0,3,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,2.28,42,64,33,17,9,28,70,31,72,0.0,3.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DW Stadium (Wigan),premier league
376,1210514400,2008-05-11 14:00:00,complete,NaN,Portsmouth,Fulham,NaN,200,1.61,0.72,1.53,0.84,0,1,1,0,0,0,NaN,76,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,2.25,45,64,31,20,11,25,58,31,70,0.0,3.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fratton Park (Portsmouth),premier league
377,1210514400,2008-05-11 14:00:00,complete,NaN,Derby County,Reading,NaN,200,0.44,0.39,0.42,0.53,0,4,4,1,0,1,NaN,"15,61,69,90'1",-1,-1,0,0,1,0,0,0,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0

# Draft on exemplary 22/23 season

In [113]:
data = pd.read_csv('./data/premier league/england-premier-league-matches-2022-to-2023-stats.csv')

In [114]:
data = clean_data(data)

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_23700\1461863474.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date_GMT'] = pd.to_datetime(data['date_GMT'])


## Variables creation

In [46]:
data['time'] = pd.to_datetime(data['time'], format='%H:%M').dt.time
data['pora dnia'] = data['time'].apply(lambda x: 'rano' if x < pd.Timestamp("12:00").time() else ('dzień' if x < pd.Timestamp("18:00").time() else 'wieczór'))

ValueError: unconverted data remains when parsing with format "%H:%M": ":00", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [49]:
data['date'] = pd.to_datetime(data['date'])
data['runda'] = data['date'].dt.month.apply(lambda x: 'wiosna' if x >= 1 and x <= 6 else 'jesień')

In [50]:
data

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,runda
0,25286.0,Crystal Palace,Arsenal,Anthony Taylor,1,0.00,0.00,1.47,2.05,0,2,2,1,0,1,3,5,1,0,2,0,0,1,1,1,11,11,3,3,8,8,16,11,56,44,0.00,0.00,1.35,1.27,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.29,3.49,2.09,1.29,1.90,3.25,6.75,1.80,1.95,Selhurst Park (London),premier league,2022-08-05,19:00:00,8,3,0,3,22,6,16,27,2,22/23,jesień
1,NaN,Fulham,Liverpool,Andy Madley,1,0.00,0.00,1.53,1.21,2,2,4,1,1,0,4,4,2,0,0,0,1,1,0,0,10,12,4,4,6,8,7,9,33,67,0.00,0.00,1.23,1.50,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,9.05,5.40,1.34,1.15,1.66,2.43,4.10,1.91,1.91,Craven Cottage (London),premier league,2022-08-06,11:30:00,8,2,0,2,22,8,14,16,0,22/23,jesień
2,NaN,AFC Bournemouth,Aston Villa,Peter Bankes,1,0.00,0.00,1.16,1.21,2,0,2,1,1,0,5,5,3,0,3,0,2,1,2,1,6,10,4,2,2,8,18,16,34,66,0.00,0.00,0.80,1.08,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.10,3.53,2.30,1.35,1.95,3.66,7.50,1.80,1.95,Vitality Stadium (Bournemouth- Dorset),premier league,2022-08-06,14:00:00,10,6,0,6,16,6,10,34,1,22/23,jesień
3,NaN,Leeds United,Wolverhampton Wanderers,Robert Jones,1,0.00,0.00,1.16,0.58,2,1,3,2,1,1,6,4,2,0,0,0,1,1,0,0,14,16,4,7,10,9,13,9,40,60,0.00,0.00,1.51,1.96,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,2.30,3.58,3.07,1.34,2.03,3.62,7.50,1.80,1.95,Elland Road (Leeds- West Yorkshire),premier league,2022-08-06,14:00:00,10,2,0,2,30,11,19,22,1,22/23,jesień
4,52245.0,Newcastle United,Nottingham Forest,Simon Hooper,1,0.00,0.00,2.05,0.42,2,0,2,0,0,0,11,1,0,0,3,0,0,0,2,1,24,6,10,2,14,4,9,14,61,39,0.00,0.00,2.93,0.71,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,1.68,3.76,4.97,1.35,1.94,3.73,7.50,2.00,1.75,St. James' Park (Newcastle upon Tyne),premier league,2022-08-06,14:00:00,12,3,0,3,30,12,18,23,1,22/23,jesień
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,NaN,Everton,AFC Bournemouth,Stuart Attwell,38,1.00,0.94,1.11,0.89,1,0,1,0,0,0,9,3,1,0,3,0,0,1,0,3,14,8,7,3,7,5,11,13,38,62,1.56,1.04,1.75,1.03,2.81,39,73,47,28,20,31,70,44,84,8.39,4.05,1.50,4.33,7.00,1.22,1.67,2.63,4.50,1.91,1.91,Goodison Park (Liverpool),premier league,2023-05-28,15:30:00,12,4,0,4,22,10,12,24,1,22/23,wiosna
376,NaN,Leeds United,Tottenham Hotspur,Anthony Taylor,38,1.22,1.11,1.16,1.21,1,4,5,1,0,1,12,3,3,0,0,0,2,1,0,0,20,12,3,8,17,4,7,5,52,48,1.59,1.48,2.08,1.53,3.45,67,81,61,50,28,42,78,59,84,10.11,4.33,2.65,3.60,2.50,1.15,1.44,2.30,3

In [51]:
data['runda'].value_counts()

runda
wiosna    216
jesień    164
Name: count, dtype: int64

In [52]:
380/2

190.0

In [53]:
data.sort_values(by=['date'], inplace=True)

# Użycie funkcji rolling() wymaga uporządkowanego indeksu
data.reset_index(drop=True, inplace=True)

# Agregacja dla całej ligi w 5 ostatnich meczach
rolling_stats_last_5 = data.rolling(window=5).mean()

# Agregacja dla całej ligi na sezon
seasonal_stats = data.groupby('season').mean()


DataError: Cannot aggregate non-numeric type: object

In [54]:
data['avg_goals_this_season'] = data.groupby('season')['total_goal_count'].transform('mean')
data['avg_home_goals_this_season'] = data.groupby('season')['home_team_goal_count'].transform('mean')
data['avg_away_goals_this_season'] = data.groupby('season')['away_team_goal_count'].transform('mean')

In [55]:
data['avg_goals_this_season']

0      2.852632
1      2.852632
2      2.852632
3      2.852632
4      2.852632
         ...   
375    2.852632
376    2.852632
377    2.852632
378    2.852632
379    2.852632
Name: avg_goals_this_season, Length: 380, dtype: float64

In [56]:
data['home_clean_sheets'] = (data['away_team_goal_count'] == 0).astype(int).cumsum()
data['away_clean_sheets'] = (data['home_team_goal_count'] == 0).astype(int).cumsum()

In [57]:
data['home_clean_sheets']

0        0
1        0
2        1
3        1
4        2
      ... 
375    124
376    124
377    125
378    125
379    125
Name: home_clean_sheets, Length: 380, dtype: int32

In [58]:
data['home_goal_difference'] = (data['home_team_goal_count'] - data['away_team_goal_count']).cumsum()
data['away_goal_difference'] = (data['away_team_goal_count'] - data['home_team_goal_count']).cumsum()

In [60]:
data['home_win'] = (data['home_team_goal_count'] > data['away_team_goal_count']).astype(int)
data['home_draw'] = (data['home_team_goal_count'] == data['away_team_goal_count']).astype(int)
data['home_loss'] = (data['home_team_goal_count'] < data['away_team_goal_count']).astype(int)

data['away_win'] = (data['away_team_goal_count'] > data['home_team_goal_count']).astype(int)
data['away_draw'] = (data['home_team_goal_count'] == data['away_team_goal_count']).astype(int)
data['away_loss'] = (data['away_team_goal_count'] < data['home_team_goal_count']).astype(int)

data['home_points'] = data['home_win']*3 + data['home_draw']*1
data['away_points'] = data['away_win']*3 + data['away_draw']*1

In [62]:
data

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,runda,avg_goals_this_season,avg_home_goals_this_season,avg_away_goals_this_season,home_clean_sheets,away_clean_sheets,home_goal_difference,away_goal_difference,home_win,home_draw,home_loss,away_win,away_draw,away_loss,home_points,away_points
0,25286.0,Crystal Palace,Arsenal,Anthony Taylor,1,0.00,0.00,1.47,2.05,0,2,2,1,0,1,3,5,1,0,2,0,0,1,1,1,11,11,3,3,8,8,16,11,56,44,0.00,0.00,1.35,1.27,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.29,3.49,2.09,1.29,1.90,3.25,6.75,1.80,1.95,Selhurst Park (London),premier league,2022-08-05,19:00:00,8,3,0,3,22,6,16,27,2,22/23,jesień,2.852632,1.634211,1.218421,0,1,-2,2,0,0,1,1,0,0,0,3
1,NaN,Fulham,Liverpool,Andy Madley,1,0.00,0.00,1.53,1.21,2,2,4,1,1,0,4,4,2,0,0,0,1,1,0,0,10,12,4,4,6,8,7,9,33,67,0.00,0.00,1.23,1.50,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,9.05,5.40,1.34,1.15,1.66,2.43,4.10,1.91,1.91,Craven Cottage (London),premier league,2022-08-06,11:30:00,8,2,0,2,22,8,14,16,0,22/23,jesień,2.852632,1.634211,1.218421,0,1,-2,2,0,1,0,0,1,0,1,1
2,NaN,AFC Bournemouth,Aston Villa,Peter Bankes,1,0.00,0.00,1.16,1.21,2,0,2,1,1,0,5,5,3,0,3,0,2,1,2,1,6,10,4,2,2,8,18,16,34,66,0.00,0.00,0.80,1.08,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.10,3.53,2.30,1.35,1.95,3.66,7.50,1.80,1.95,Vitality Stadium (Bournemouth- Dorset),premier league,2022-08-06,14:00:00,10,6,0,6,16,6,10,34,1,22/23,jesień,2.852632,1.634211,1.218421,1,1,0,0,1,0,0,0,0,1,3,0
3,NaN,Leeds United,Wolverhampton Wanderers,Robert Jones,1,0.00,0.00,1.16,0.58,2,1,3,2,1,1,6,4,2,0,0,0,1,1,0,0,14,16,4,7,10,9,13,9,40,60,0.00,0.00,1.51,1.96,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,2.30,3.58,3.07,1.34,2.03,3.62,7.50,1.80,1.95,Elland Road (Leeds- West Yorkshire),premier league,2022-08-06,14:00:00,10,2,0,2,30,11,19,22,1,22/23,jesień,2.852632,1.634211,1.218421,1,1,1,-1,1,0,0,0,0,1,3,0
4,52245.0,Newcastle United,Nottingham Forest,Simon Hooper,1,0.00,0.00,2.05,0.42,2,0,2,0,0,0,11,1,0,0,3,0,0,0,2,1,24,6,10,2,14,4,9,14,61,39,0.00,0.00,2.93,0.71,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,1.68,3.76,4.97,1.35,1.94,3.73,7.50,2.00,1.75,St. James' Park (Newcastle upon Tyne),premier league,2022-08-06,14:00:00,12,3,0,3,30,12,18,23,1,22/23,jesień,2.852632,1.634211,1.218421,2,1,3,-3,1,0,0,0,0,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,N

In [64]:
# Dodaj kolumny dla bilansu bramkowego dla każdej drużyny osobno, zarówno dla gospodarzy, jak i gości
data['home_goal_difference'] = data.groupby('home_team_name')['home_team_goal_count'].cumsum() - data.groupby('home_team_name')['away_team_goal_count'].cumsum()
data['away_goal_difference'] = data.groupby('away_team_name')['away_team_goal_count'].cumsum() - data.groupby('away_team_name')['home_team_goal_count'].cumsum()

In [75]:
data.loc[data['Game Week'] == 10]

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,runda,avg_goals_this_season,avg_home_goals_this_season,avg_away_goals_this_season,home_clean_sheets,away_clean_sheets,home_goal_difference,away_goal_difference,home_win,home_draw,home_loss,away_win,away_draw,away_loss,home_points,away_points,home_cumulative_goal_diff,away_cumulative_goal_diff,home_goals_scored,home_goals_conceded,away_goals_scored,away_goals_conceded,cumulative_goals_scored,cumulative_goals_conceded,home_goals_scored_cum,home_goals_conceded_cum,away_goals_scored_cum,away_goals_conceded_cum,cumulative_goal_difference
77,10272.0,AFC Bournemouth,Leicester City,Michael Salisbury,10,1.25,0.00,1.16,0.79,2,1,3,1,0,1,7,2,1,0,2,0,1,0,0,2,10,10,6,5,4,5,8,16,44,56,0.81,1.67,1.48,1.27,3.63,50,75,63,38,38,50,63,50,75,8.00,3.50,3.44,3.55,2.14,1.27,1.76,3.05,6.20,1.67,2.10,Vitality Stadium (Bournemouth- Dorset),premier league,2022-10-08,14:00:00,9,3,0,3,20,11,9,24,1,22/23,jesień,2.852632,1.634211,1.218421,22,18,0,-11,1,0,0,0,0,1,3,0,-1,-10,2,3,7,17,7.0,14.0,2,3,7,17,-7.0
78,39940.0,Chelsea,Wolverhampton Wanderers,Simon Hooper,10,2.33,0.25,1.32,0.58,3,0,3,1,1,0,4,2,2,0,0,0,1,1,0,0,20,9,8,3,12,6,8,8,57,43,1.51,1.91,2.21,1.10,2.42,63,75,63,17,0,13,42,50,88,12.25,4.58,1.40,4.85,8.37,1.28,1.84,3.20,6.20,2.20,1.62,Stamford Bridge (London),premier league,2022-10-08,14:00:00,6,2,0,2,29,11,18,16,1,22/23,jesień,2.852632,1.634211,1.218421,23,18,5,-7,1,0,0,0,0,1,3,0,2,-4,6,4,1,5,5.0,6.0,6,4,1,5,-1.0
79,53365.0,Manchester City,Southampton,Andy Madley,10,3.00,0.75,2.74,0.74,4,0,4,2,2,0,9,1,0,0,1,0,0,0,1,0,22,6,9,0,13,6,4,8,65,35,2.39,1.11,2.46,0.59,4.38,50,75,75,63,50,63,88,63,75,12.00,1.75,1.13,9.60,19.66,1.08,1.26,1.65,2.78,2.00,1.75,Etihad Stadium (Manchester),premier league,2022-10-08,14:00:00,10,1,0,1,28,9,19,12,1,22/23,jesień,2.852632,1.634211,1.218421,24,18,19,-8,1,0,0,0,0,1,3,0,15,-4,20,5,3,7,17.0,8.0,20,5,3,7,9.0
80,52067.0,Newcastle United,Brentford,John Brooks,10,1.50,0.75,2.05,1.16,5,1,6,2,2,0,9,5,0,0,1,0,0,0,1,0,17,9,8,5,9,4,14,9,54,46,2.41,1.47,1.97,1.26,2.63,63,75,38,38,25,25,38,75,75,15.00,3.50,1.79,3.85,4.51,1.26,1.80,3.00,5.80,1.70,2.05,St. James' Park (Newcastle upon Tyne),premier league,2022-10-08,14:00:00,14,1,0,1,26,13,13,23,1,22/23,jesień,2.852632,1.634211,1.218421,24,18,6,-5,1,0,0,0,0,1,3,0,2,-1,6,4,5,6,10.0,9.0,6,4,5,6,1.0
81,31498.0,Brighton & Hove Albion,Tottenham Hotspur,Tony Harrington,10,2.33,1.25,1.79,1.21,0,1,1,1,0,1,6,2,1,0,2,0,0,1,2,0,15,9,5,4,10,5,9,14,59,41,2.02,1.38,1.81,1.06,2.84,54,67,42,42,17,29,67,42,84,10.33,3.92,3.25,3.45,2.12,1.33,1.78,

In [68]:
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values(by='date')

In [73]:
# Inicjalizacja kolumn dla skumulowanych bramek zdobytych i straconych
data['cumulative_goals_scored'] = 0
data['cumulative_goals_conceded'] = 0

# Obliczenia dla gospodarzy
data['home_goals_scored_cum'] = data.groupby('home_team_name')['home_team_goal_count'].cumsum() - data['home_team_goal_count']
data['home_goals_conceded_cum'] = data.groupby('home_team_name')['away_team_goal_count'].cumsum() - data['away_team_goal_count']

# Obliczenia dla gości
data['away_goals_scored_cum'] = data.groupby('away_team_name')['away_team_goal_count'].cumsum() - data['away_team_goal_count']
data['away_goals_conceded_cum'] = data.groupby('away_team_name')['home_team_goal_count'].cumsum() - data['home_team_goal_count']

# Łączenie wyników gospodarzy i gości
data['cumulative_goals_scored'] = data.groupby('home_team_name')['home_goals_scored_cum'].shift(1) + data.groupby('away_team_name')['away_goals_scored_cum'].shift(1)
data['cumulative_goals_conceded'] = data.groupby('home_team_name')['home_goals_conceded_cum'].shift(1) + data.groupby('away_team_name')['away_goals_conceded_cum'].shift(1)

# Uzupełnienie braków danych spowodowanych przesunięciem
data['cumulative_goals_scored'].fillna(0, inplace=True)
data['cumulative_goals_conceded'].fillna(0, inplace=True)

# Obliczenie skumulowanego bilansu bramek
data['cumulative_goal_difference'] = data['cumulative_goals_scored'] - data['cumulative_goals_conceded']

In [74]:
data

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,runda,avg_goals_this_season,avg_home_goals_this_season,avg_away_goals_this_season,home_clean_sheets,away_clean_sheets,home_goal_difference,away_goal_difference,home_win,home_draw,home_loss,away_win,away_draw,away_loss,home_points,away_points,home_cumulative_goal_diff,away_cumulative_goal_diff,home_goals_scored,home_goals_conceded,away_goals_scored,away_goals_conceded,cumulative_goals_scored,cumulative_goals_conceded,home_goals_scored_cum,home_goals_conceded_cum,away_goals_scored_cum,away_goals_conceded_cum,cumulative_goal_difference
0,25286.0,Crystal Palace,Arsenal,Anthony Taylor,1,0.00,0.00,1.47,2.05,0,2,2,1,0,1,3,5,1,0,2,0,0,1,1,1,11,11,3,3,8,8,16,11,56,44,0.00,0.00,1.35,1.27,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.29,3.49,2.09,1.29,1.90,3.25,6.75,1.80,1.95,Selhurst Park (London),premier league,2022-08-05,19:00:00,8,3,0,3,22,6,16,27,2,22/23,jesień,2.852632,1.634211,1.218421,0,1,-2,2,0,0,1,1,0,0,0,3,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0.0
1,NaN,Fulham,Liverpool,Andy Madley,1,0.00,0.00,1.53,1.21,2,2,4,1,1,0,4,4,2,0,0,0,1,1,0,0,10,12,4,4,6,8,7,9,33,67,0.00,0.00,1.23,1.50,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,9.05,5.40,1.34,1.15,1.66,2.43,4.10,1.91,1.91,Craven Cottage (London),premier league,2022-08-06,11:30:00,8,2,0,2,22,8,14,16,0,22/23,jesień,2.852632,1.634211,1.218421,0,1,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0.0
2,NaN,AFC Bournemouth,Aston Villa,Peter Bankes,1,0.00,0.00,1.16,1.21,2,0,2,1,1,0,5,5,3,0,3,0,2,1,2,1,6,10,4,2,2,8,18,16,34,66,0.00,0.00,0.80,1.08,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.10,3.53,2.30,1.35,1.95,3.66,7.50,1.80,1.95,Vitality Stadium (Bournemouth- Dorset),premier league,2022-08-06,14:00:00,10,6,0,6,16,6,10,34,1,22/23,jesień,2.852632,1.634211,1.218421,1,1,2,-2,1,0,0,0,0,1,3,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0.0
3,NaN,Leeds United,Wolverhampton Wanderers,Robert Jones,1,0.00,0.00,1.16,0.58,2,1,3,2,1,1,6,4,2,0,0,0,1,1,0,0,14,16,4,7,10,9,13,9,40,60,0.00,0.00,1.51,1.96,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,2.30,3.58,3.07,1.34,2.03,3.62,7.50,1.80,1.95,Elland Road (Leeds- West Yorkshire),premier league,2022-08-06,14:00:00,10,2,0,2,30,11,19,22,1,22/23,jesień,2.852632,1.634211,1.218421,1,1,1,-1,1,0,0,0,0,1,3,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0.0
4,52245.0,Newcastle United,Nottingham Forest,Simon Hooper,1,0.00,0.00,2.05,0.42,2,0,2,0,0,0,11,1,0,0,3,0,0,0,2,1,24,6,10,2,14,4,9,14,61,39,0.00,0.00,2.93,0.71,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,1.68,3.76,4.97,1.35,1.94,3.73,7.50,2.00,1.75,St. James' Park (Newcastle upon Tyne),premier league,2022-08-06,14:00:00,12,3,0,3,30,12,18,23,1,22/23,jes

In [72]:
data[(data['home_team_name'] == 'Arsenal') | (data['away_team_name'] == 'Arsenal')][['home_team_name', 'away_team_name']]

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,runda,avg_goals_this_season,avg_home_goals_this_season,avg_away_goals_this_season,home_clean_sheets,away_clean_sheets,home_goal_difference,away_goal_difference,home_win,home_draw,home_loss,away_win,away_draw,away_loss,home_points,away_points,home_cumulative_goal_diff,away_cumulative_goal_diff,home_goals_scored,home_goals_conceded,away_goals_scored,away_goals_conceded
0,25286.0,Crystal Palace,Arsenal,Anthony Taylor,1,0.00,0.00,1.47,2.05,0,2,2,1,0,1,3,5,1,0,2,0,0,1,1,1,11,11,3,3,8,8,16,11,56,44,0.00,0.00,1.35,1.27,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.29,3.49,2.09,1.29,1.90,3.25,6.75,1.80,1.95,Selhurst Park (London),premier league,2022-08-05,19:00:00,8,3,0,3,22,6,16,27,2,22/23,jesień,2.852632,1.634211,1.218421,0,1,-2,2,0,0,1,1,0,0,0,3,0,0,0,0,0,0
13,60033.0,Arsenal,Leicester City,Darren England,2,0.00,0.00,2.37,0.79,4,2,6,2,2,0,6,2,1,0,1,0,0,1,1,0,20,8,8,4,12,4,15,9,50,50,0.00,0.00,2.29,0.99,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,1.68,3.50,4.07,1.23,1.86,2.79,5.20,1.80,1.95,Emirates Stadium (London),premier league,2022-08-13,14:00:00,8,2,0,2,28,12,16,24,1,22/23,jesień,2.852632,1.634211,1.218421,4,4,2,-2,1,0,0,0,0,1,3,0,0,0,0,0,0,0
21,10423.0,AFC Bournemouth,Arsenal,Craig Pawson,3,3.00,3.00,1.16,2.05,0,3,3,2,0,2,3,4,2,0,1,0,1,1,0,1,7,14,3,6,4,8,12,10,42,58,0.80,1.27,0.90,1.68,2.00,0,100,0,0,0,0,100,0,100,10.00,5.00,5.75,3.90,1.60,1.26,1.80,3.00,5.90,2.00,1.75,Vitality Stadium (Bournemouth- Dorset),premier league,2022-08-20,16:30:00,7,3,0,3,21,9,12,22,2,22/23,jesień,2.852632,1.634211,1.218421,7,6,-1,5,0,0,1,1,0,0,0,3,2,2,2,0,2,0
30,60164.0,Arsenal,Fulham,Jarred Gillett,4,3.00,1.00,2.37,1.21,2,1,3,0,0,0,9,2,2,0,3,0,2,0,1,2,23,12,9,4,14,8,6,12,72,28,2.29,0.93,2.65,1.19,3.00,50,50,50,50,50,50,50,50,50,9.00,4.00,1.33,5.40,8.60,1.16,1.57,2.30,4.05,1.80,1.95,Emirates Stadium (London),premier league,2022-08-27,16:30:00,11,5,0,5,35,13,22,18,1,22/23,jesień,2.852632,1.634211,1.218421,9,7,3,-1,1,0,0,0,0,1,3,0,2,0,4,2,0,0
47,60012.0,Arsenal,Aston Villa,Robert Jones,5,3.00,0.00,2.37,1.21,2,1,3,1,1,0,10,3,2,0,3,0,1,1,1,2,23,5,9,4,14,1,10,14,59,41,2.47,1.34,2.55,0.83,3.75,75,100,75,50,25,50,75,75,100,11.00,3.50,1.46,4.60,7.30,1.25,1.67,2.30,5.60,1.95,1.80,Emirates Stadium (London),premier league,2022-08-31,18:30:00,13,5,0,5,28,13,15,24,1,22/23,jesień,2.852632,1.634211,1.218421,12,10,4,-5,1,0,0,0,0,1,3,0,3,-4,6,3,1,5
58,NaN,Manchester United,Arsenal,Paul Tierney,6,1.50,3.00,2.53,2.05,3,1,4,1,1,0,2,5,3,0,3,0,0,3,1,2,10,17,7,4,3,13,13,9,40,60,1.72,1.47,1.35,1.92,2.75,50,100,75,0,0,50,100,25,100,10.

In [76]:
# Tworzenie kolumn do zbierania danych dla drużyn u siebie i na wyjeździe
data['goals_scored'] = data['home_team_goal_count'].where(data['home_team_name'] == data['home_team_name'], data['away_team_goal_count'])
data['goals_conceded'] = data['away_team_goal_count'].where(data['home_team_name'] == data['home_team_name'], data['home_team_goal_count'])

# Skumulowane sumy bramek zdobytych i straconych dla każdej drużyny
data['cumulative_goals_scored'] = data.groupby('home_team_name')['goals_scored'].cumsum() + data.groupby('away_team_name')['goals_scored'].cumsum()
data['cumulative_goals_conceded'] = data.groupby('home_team_name')['goals_conceded'].cumsum() + data.groupby('away_team_name')['goals_conceded'].cumsum()

# Obliczanie skumulowanego bilansu bramek dla każdej drużyny
data['cumulative_goal_difference'] = data['cumulative_goals_scored'] - data['cumulative_goals_conceded']

In [77]:
# Usunięcie bieżącej kolejki z bilansu
data['cumulative_goals_scored'] -= data['goals_scored']
data['cumulative_goals_conceded'] -= data['goals_conceded']
data['cumulative_goal_difference'] = data['cumulative_goals_scored'] - data['cumulative_goals_conceded']

# Mapowanie wartości bilansu dla gospodarza i gościa
data['home_goal_difference_before_match'] = data['cumulative_goal_difference'].where(data['home_team_name'] == data['home_team_name'])
data['away_goal_difference_before_match'] = data['cumulative_goal_difference'].where(data['away_team_name'] == data['away_team_name'])

In [78]:
data.drop(['goals_scored', 'goals_conceded', 'cumulative_goals_scored', 'cumulative_goals_conceded', 'cumulative_goal_difference'], axis=1, inplace=True)

In [79]:
data

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,runda,avg_goals_this_season,avg_home_goals_this_season,avg_away_goals_this_season,home_clean_sheets,away_clean_sheets,home_goal_difference,away_goal_difference,home_win,home_draw,home_loss,away_win,away_draw,away_loss,home_points,away_points,home_cumulative_goal_diff,away_cumulative_goal_diff,home_goals_scored,home_goals_conceded,away_goals_scored,away_goals_conceded,home_goals_scored_cum,home_goals_conceded_cum,away_goals_scored_cum,away_goals_conceded_cum,home_goal_difference_before_match,away_goal_difference_before_match
0,25286.0,Crystal Palace,Arsenal,Anthony Taylor,1,0.00,0.00,1.47,2.05,0,2,2,1,0,1,3,5,1,0,2,0,0,1,1,1,11,11,3,3,8,8,16,11,56,44,0.00,0.00,1.35,1.27,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.29,3.49,2.09,1.29,1.90,3.25,6.75,1.80,1.95,Selhurst Park (London),premier league,2022-08-05,19:00:00,8,3,0,3,22,6,16,27,2,22/23,jesień,2.852632,1.634211,1.218421,0,1,-2,2,0,0,1,1,0,0,0,3,0,0,0,0,0,0,0,0,0,0,-2,-2
1,NaN,Fulham,Liverpool,Andy Madley,1,0.00,0.00,1.53,1.21,2,2,4,1,1,0,4,4,2,0,0,0,1,1,0,0,10,12,4,4,6,8,7,9,33,67,0.00,0.00,1.23,1.50,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,9.05,5.40,1.34,1.15,1.66,2.43,4.10,1.91,1.91,Craven Cottage (London),premier league,2022-08-06,11:30:00,8,2,0,2,22,8,14,16,0,22/23,jesień,2.852632,1.634211,1.218421,0,1,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,NaN,AFC Bournemouth,Aston Villa,Peter Bankes,1,0.00,0.00,1.16,1.21,2,0,2,1,1,0,5,5,3,0,3,0,2,1,2,1,6,10,4,2,2,8,18,16,34,66,0.00,0.00,0.80,1.08,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.10,3.53,2.30,1.35,1.95,3.66,7.50,1.80,1.95,Vitality Stadium (Bournemouth- Dorset),premier league,2022-08-06,14:00:00,10,6,0,6,16,6,10,34,1,22/23,jesień,2.852632,1.634211,1.218421,1,1,2,-2,1,0,0,0,0,1,3,0,0,0,0,0,0,0,0,0,0,0,2,2
3,NaN,Leeds United,Wolverhampton Wanderers,Robert Jones,1,0.00,0.00,1.16,0.58,2,1,3,2,1,1,6,4,2,0,0,0,1,1,0,0,14,16,4,7,10,9,13,9,40,60,0.00,0.00,1.51,1.96,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,2.30,3.58,3.07,1.34,2.03,3.62,7.50,1.80,1.95,Elland Road (Leeds- West Yorkshire),premier league,2022-08-06,14:00:00,10,2,0,2,30,11,19,22,1,22/23,jesień,2.852632,1.634211,1.218421,1,1,1,-1,1,0,0,0,0,1,3,0,0,0,0,0,0,0,0,0,0,0,1,1
4,52245.0,Newcastle United,Nottingham Forest,Simon Hooper,1,0.00,0.00,2.05,0.42,2,0,2,0,0,0,11,1,0,0,3,0,0,0,2,1,24,6,10,2,14,4,9,14,61,39,0.00,0.00,2.93,0.71,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,1.68,3.76,4.97,1.35,1.94,3.73,7.50,2.00,1.75,St. James' Park (Newcastle upon Tyne),premier league,2022-08-06,14:00:00,12,3,0,3,30,12,18,23,1,22/23,jesień,2.852632,1.634211,1.218421,2,1,2,-2

In [80]:
# Inicjalizacja kolumn dla skumulowanego bilansu bramek
data['home_goals_scored_cum'] = data.groupby('home_team_name')['home_team_goal_count'].cumsum() - data['home_team_goal_count']
data['home_goals_conceded_cum'] = data.groupby('home_team_name')['away_team_goal_count'].cumsum() - data['away_team_goal_count']
data['away_goals_scored_cum'] = data.groupby('away_team_name')['away_team_goal_count'].cumsum() - data['away_team_goal_count']
data['away_goals_conceded_cum'] = data.groupby('away_team_name')['home_team_goal_count'].cumsum() - data['home_team_goal_count']

# Łączenie bilansów zdobytych i straconych bramek w domu i na wyjeździe dla gospodarzy i gości
data['home_goal_diff'] = data.groupby('home_team_name')['home_goals_scored_cum'].shift(1) - data.groupby('home_team_name')['home_goals_conceded_cum'].shift(1)
data['away_goal_diff'] = data.groupby('away_team_name')['away_goals_scored_cum'].shift(1) - data.groupby('away_team_name')['away_goals_conceded_cum'].shift(1)


In [81]:
# Przypisanie obliczonego bilansu dla gospodarzy i gości do rekordów meczów
data['home_goal_difference_before_match'] = data['home_goal_diff']
data['away_goal_difference_before_match'] = data['away_goal_diff']


In [84]:
data

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league
0,1659726000,Aug 05 2022 - 7:00pm,complete,25286.0,Crystal Palace,Arsenal,Anthony Taylor,1,0.00,0.00,1.47,2.05,0,2,2,1,0,1,NaN,"20,85",3,5,1,0,2,0,0,1,1,1,11,11,3,3,8,8,16,11,56,44,0.00,0.00,1.35,1.27,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.29,3.49,2.09,1.29,1.90,3.25,6.75,1.80,1.95,Selhurst Park (London),premier league
1,1659785400,Aug 06 2022 - 11:30am,complete,NaN,Fulham,Liverpool,Andy Madley,1,0.00,0.00,1.53,1.21,2,2,4,1,1,0,"32,72","64,80",4,4,2,0,0,0,1,1,0,0,10,12,4,4,6,8,7,9,33,67,0.00,0.00,1.23,1.50,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,9.05,5.40,1.34,1.15,1.66,2.43,4.10,1.91,1.91,Craven Cottage (London),premier league
2,1659794400,Aug 06 2022 - 2:00pm,complete,NaN,AFC Bournemouth,Aston Villa,Peter Bankes,1,0.00,0.00,1.16,1.21,2,0,2,1,1,0,"2,80",NaN,5,5,3,0,3,0,2,1,2,1,6,10,4,2,2,8,18,16,34,66,0.00,0.00,0.80,1.08,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.10,3.53,2.30,1.35,1.95,3.66,7.50,1.80,1.95,Vitality Stadium (Bournemouth- Dorset),premier league
3,1659794400,Aug 06 2022 - 2:00pm,complete,NaN,Leeds United,Wolverhampton Wanderers,Robert Jones,1,0.00,0.00,1.16,0.58,2,1,3,2,1,1,"24,74",6,6,4,2,0,0,0,1,1,0,0,14,16,4,7,10,9,13,9,40,60,0.00,0.00,1.51,1.96,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,2.30,3.58,3.07,1.34,2.03,3.62,7.50,1.80,1.95,Elland Road (Leeds- West Yorkshire),premier league
4,1659794400,Aug 06 2022 - 2:00pm,complete,52245.0,Newcastle United,Nottingham Forest,Simon Hooper,1,0.00,0.00,2.05,0.42,2,0,2,0,0,0,"58,78",NaN,11,1,0,0,3,0,0,0,2,1,24,6,10,2,14,4,9,14,61,39,0.00,0.00,2.93,0.71,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,1.68,3.76,4.97,1.35,1.94,3.73,7.50,2.00,1.75,St. James' Park (Newcastle upon Tyne),premier league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1685287800,May 28 2023 - 3:30pm,complete,NaN,Everton,AFC Bournemouth,Stuart Attwell,38,1.00,0.94,1.11,0.89,1,0,1,0,0,0,57,NaN,9,3,1,0,3,0,0,1,0,3,14,8,7,3,7,5,11,13,38,62,1.56,1.04,1.75,1.03,2.81,39,73,47,28,20,31,70,44,84,8.39,4.05,1.50,4.33,7.00,1.22,1.67,2.63,4.50,1.91,1.91,Goodison Park (Liverpool),premier league
376,1685287800,May 28 2023 - 3:30pm,complete,NaN,Leeds United,Tottenham Hotspur,Anthony Taylor,38,1.22,1.11,1.16,1.21,1,4,5,1,0,1,67,"2,47,69,90'5",12,3,3,0,0,0,2,1,0,0,20,12,3,8,17,4,7,5,52,48,1.59,1.48,2.08,1.53,3.45,67,81,61,50,28,42,78,59,84,10.11,4.33,2.65,3.60,2.50,1.15,1.44,2.30,3.75,1.44,2.63,Elland Road (Leeds- West Yorkshire),premier league
377,16852878

In [108]:
# Obliczanie skumulowanych bramek zdobytych przez drużyny jako gospodarze
data['cumulative_home_goals'] = data.groupby('home_team_name')['home_team_goal_count'].cumsum()

# Obliczanie skumulowanych bramek zdobytych przez drużyny jako goście
data['cumulative_away_goals'] = data.groupby('away_team_name')['away_team_goal_count'].cumsum()

In [109]:
data = data.merge(data[['away_team_name', 'cumulative_away_goals']].rename(columns={'away_team_name': 'team_name', 'cumulative_away_goals': 'team_away_goals'}), left_on='home_team_name', right_on='team_name', how='left')

In [110]:
# Dołączenie skumulowanych bramek zdobytych w domu do tabeli, zmapowanie po nazwie drużyny gospodarzy
data = data.merge(data[['home_team_name', 'cumulative_home_goals']].rename(columns={'home_team_name': 'team_name', 'cumulative_home_goals': 'team_home_goals'}), left_on='away_team_name', right_on='team_name', how='left')

In [111]:
# Zsumowanie bramek zdobytych w domu i na wyjeździe dla każdej drużyny
data['total_goals_scored_by_team'] = data['team_home_goals'].fillna(0) + data['team_away_goals'].fillna(0)

In [116]:
data.groupby('home_team_name')['home_team_goal_count'].cumsum()

0       0
1       2
2       2
3       2
4       2
       ..
375    16
376    26
377    23
378    36
379    19
Name: home_team_goal_count, Length: 380, dtype: int64

In [121]:
data.groupby('away_team_name')['away_team_goal_count'].cumsum().loc[365]

17

In [120]:
data[data['away_team_name'] == 'Southampton']

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season
5,61732.0,Tottenham Hotspur,Southampton,Andre Marriner,1,0.00,0.00,1.95,0.74,4,1,5,3,2,1,10,2,3,0,0,0,1,2,0,0,15,8,9,3,6,5,11,6,58,42,0.00,0.00,1.65,0.77,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,1.58,4.42,5.60,1.18,1.63,2.61,4.80,1.95,1.80,Tottenham Hotspur Stadium (London),premier league,2022-08-06,14:00:00,12,3,0,3,23,12,11,17,1,22/23
24,NaN,Leicester City,Southampton,Michael Salisbury,3,1.00,0.00,1.00,0.74,1,2,3,0,0,0,2,5,2,0,3,0,0,2,0,3,7,10,3,4,4,6,11,15,62,38,1.42,0.77,1.14,1.25,4.50,100,100,100,100,50,50,100,100,100,7.00,0.00,1.86,4.10,4.10,1.22,1.63,2.69,5.00,1.62,2.20,King Power Stadium (Leicester- Leicestershire),premier league,2022-08-20,14:00:00,7,5,0,5,17,7,10,26,2,22/23
55,30.0,Wolverhampton Wanderers,Southampton,John Brooks,6,1.00,1.50,1.58,0.74,1,0,1,1,1,0,5,6,4,0,2,0,1,3,1,1,8,14,3,3,5,11,10,14,51,49,1.10,1.01,1.13,1.54,2.50,75,75,50,25,25,25,50,50,75,8.00,4.00,2.11,3.55,3.50,1.33,2.02,3.40,6.00,1.80,1.91,Molineux Stadium (Wolverhampton- West Midlands),premier league,2022-09-03,14:00:00,11,6,0,6,22,6,16,24,1,22/23
60,NaN,Aston Villa,Southampton,Tony Harrington,8,1.33,1.00,2.00,0.74,1,0,1,1,1,0,6,3,3,0,1,0,1,2,0,1,12,8,4,2,8,6,13,13,53,47,1.22,1.19,1.36,0.89,2.50,67,67,50,17,17,17,50,67,84,7.33,4.34,2.00,3.75,3.40,1.29,1.91,3.20,6.20,1.70,2.05,Villa Park (Birmingham),premier league,2022-09-16,19:00:00,9,4,0,4,20,6,14,26,1,22/23
79,53365.0,Manchester City,Southampton,Andy Madley,10,3.00,0.75,2.74,0.74,4,0,4,2,2,0,9,1,0,0,1,0,0,0,1,0,22,6,9,0,13,6,4,8,65,35,2.39,1.11,2.46,0.59,4.38,50,75,75,63,50,63,88,63,75,12.00,1.75,1.13,9.60,19.66,1.08,1.26,1.65,2.78,2.00,1.75,Etihad Stadium (Manchester),premier league,2022-10-08,14:00:00,10,1,0,1,28,9,19,12,1,22/23
99,NaN,AFC Bournemouth,Southampton,John Brooks,12,1.60,0.60,1.16,0.74,0,1,1,1,0,1,7,3,1,0,3,0,1,0,2,1,17,9,5,4,12,5,8,13,53,47,0.94,1.01,1.93,1.17,2.20,30,60,50,20,10,30,70,40,60,8.20,3.20,2.75,3.10,2.60,1.30,2.15,3.20,6.25,1.75,2.00,Vitality Stadium (Bournemouth- Dorset),premier league,2022-10-19,18:30:00,10,4,0,4,26,9,17,21,2,22/23
120,NaN,Crystal Palace,Southampton,Michael Salisbury,14,1.67,1.00,1.47,0.74,1,0,1,1,1,0,8,4,3,0,3,0,1,2,3,0,15,16,5,6,10,10,13,12,45,55,1.59,1.03,1.76,1.82,2.67,58,75,59,25,9,42,83,50,75,7.50,3.50,1.95,3.50,3.90,1.35,1.98,3.75,7.60,1.91,1.91,Selhurst Park (London),premier league,2022-10-29,14:00:00,12,6,0,6,31,11,20,25,1,22/23
138,NaN,Liverpool,Southampton,Simon Hooper,16,2.00,0.86,2.32,0.74,3,1,4,4,3,1,6,0,0,0,2,0,0,0,1,1,16,10,8,6,8,4,8,8,60,40,2.44,1.15,1.98,1.40,2.93,43,57,50,29,

# Brudnopis

In [49]:
df = pd.read_csv('data\premier league\england-premier-league-matches-2022-to-2023-stats.csv')

In [50]:
df = clean_data(df)

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_2548\1461863474.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date_GMT'] = pd.to_datetime(data['date_GMT'])


In [51]:
df.sort_values('date', inplace=True)

In [53]:
# to jest elegancko - teraz to musze wykorzystać do pozostałych agregatów

# Tworzenie DataFrame z bramkami zdobywanymi zarówno w domu, jak i na wyjeździe
goals = pd.concat([
    df[['date', 'home_team_name', 'home_team_goal_count']].rename(columns={'home_team_name': 'team', 'home_team_goal_count': 'goals'}),
    df[['date', 'away_team_name', 'away_team_goal_count']].rename(columns={'away_team_name': 'team', 'away_team_goal_count': 'goals'})
])

# Sortowanie danych według daty, aby kumulatywne sumy były poprawne
goals.sort_values('date', inplace=True)

# Obliczanie kumulatywnej liczby bramek dla każdej drużyny
goals['cumulative_goals'] = goals.groupby('team')['goals'].cumsum()

# Usunięcie bieżących bramek z kumulatywnej sumy, aby liczyć tylko bramki przed bieżącym meczem
goals['cumulative_goals'] -= goals['goals']

# Powrót do oryginalnego DataFrame z wynikami meczów i dodanie obliczonych kumulatywnych bramek
df = df.merge(goals[['date', 'team', 'cumulative_goals']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_goals': 'cumulative_home_goals'}).drop('team', axis=1)
df = df.merge(goals[['date', 'team', 'cumulative_goals']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_goals': 'cumulative_away_goals'}).drop('team', axis=1)


In [54]:
df.shape

(380, 76)

In [59]:
df[(df['home_team_name'] == "Fulham") | (df['away_team_name'] == "Fulham")][['home_team_name', 'away_team_name', 'home_team_goal_count', 'away_team_goal_count', 'cumulative_home_goals', 'cumulative_away_goals']]

,home_team_name,away_team_name,home_team_goal_count,away_team_goal_count,cumulative_home_goals,cumulative_away_goals
1,Fulham,Liverpool,2,2,0,0
11,Wolverhampton Wanderers,Fulham,0,0,1,2
22,Fulham,Brentford,3,2,2,6
30,Arsenal,Fulham,2,1,9,5
41,Fulham,Brighton & Hove Albion,2,1,6,5
54,Tottenham Hotspur,Fulham,2,1,10,8
61,Nottingham Forest,Fulham,2,3,4,9
70,Fulham,Newcastle United,1,4,12,8
85,West Ham United,Fulham,3,1,5,13
89,Fulham,AFC Bournemouth,2,2,14,8


In [41]:
total_goals = pd.concat([
    data[['home_team_name', 'home_team_goal_count']].rename(columns={'home_team_name': 'team', 'home_team_goal_count': 'goals'}),
    data[['away_team_name', 'away_team_goal_count']].rename(columns={'away_team_name': 'team', 'away_team_goal_count': 'goals'})
]).reset_index(drop=True)

In [42]:
total_goals

,team,goals
0,Crystal Palace,0
1,Fulham,2
2,AFC Bournemouth,2
3,Leeds United,2
4,Newcastle United,2
...,...,...
755,Manchester City,0
756,Brighton & Hove Albion,1
757,Wolverhampton Wanderers,0
758,Newcastle United,1


In [37]:
# Obliczanie kumulatywnej sumy bramek dla każdej drużyny
total_goals['cumulative_goals'] = total_goals.groupby('team')['goals'].cumsum() - total_goals['goals']

In [38]:
# Tworzenie odwzorowania kumulatywnych bramek do odpowiednich drużyn
cumulative_goals_by_team = total_goals.groupby('team')['cumulative_goals'].shift().fillna(0)

In [45]:
# Tworzenie pomocniczego DataFrame do przechowywania wszystkich bramek dla każdej drużyny
total_goals = pd.concat([
    data[['home_team_name', 'home_team_goal_count']].rename(columns={'home_team_name': 'team', 'home_team_goal_count': 'goals'}),
    data[['away_team_name', 'away_team_goal_count']].rename(columns={'away_team_name': 'team', 'away_team_goal_count': 'goals'})
]).reset_index(drop=True)

# Obliczanie kumulatywnej sumy bramek dla każdej drużyny
total_goals['cumulative_goals'] = total_goals.groupby('team')['goals'].cumsum() - total_goals['goals']

# Przesunięcie kumulatywnych wyników, aby nie uwzględniały bieżącego meczu
total_goals['shifted_cumulative_goals'] = total_goals.groupby('team')['cumulative_goals'].shift().fillna(0)

# Przekształcenie danych do słownika, który zostanie użyty do mapowania
cumulative_goals_dict = total_goals.groupby('team')['shifted_cumulative_goals'].last().to_dict()

# Przypisanie kumulatywnej sumy bramek do meczów, w zależności od tego, czy drużyna jest gospodarzem czy gościem
data['cumulative_home_goals'] = data['home_team_name'].map(cumulative_goals_dict)
data['cumulative_away_goals'] = data['away_team_name'].map(cumulative_goals_dict)

In [47]:
data[['date', 'home_team_name', 'away_team_name', 'cumulative_home_goals', 'cumulative_away_goals']]

,date,home_team_name,away_team_name,cumulative_home_goals,cumulative_away_goals
0,2022-08-05,Crystal Palace,Arsenal,38.0,86.0
1,2022-08-06,Fulham,Liverpool,52.0,68.0
2,2022-08-06,AFC Bournemouth,Aston Villa,37.0,50.0
3,2022-08-06,Leeds United,Wolverhampton Wanderers,46.0,31.0
4,2022-08-06,Newcastle United,Nottingham Forest,65.0,35.0
...,...,...,...,...,...
372,2023-05-28,Brentford,Manchester City,55.0,93.0
371,2023-05-28,Aston Villa,Brighton & Hove Albion,50.0,70.0
370,2023-05-28,Arsenal,Wolverhampton Wanderers,86.0,31.0
373,2023-05-28,Chelsea,Newcastle United,37.0,65.0


In [30]:
data['cumulative_home_goals'] = data['home_team_name'].map(total_goals.groupby('team')['cumulative_goals'].last())
data['cumulative_away_goals'] = data['away_team_name'].map(total_goals.groupby('team')['cumulative_goals'].last())

In [34]:
data[data['Game Week'] == 2]

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals,cumulative_away_goals
16,NaN,Brentford,Manchester United,Stuart Attwell,2,0.0,0.0,1.95,1.42,4,0,4,4,4,0,8,2,0,0,4,0,0,0,1,3,14,16,8,5,6,11,6,15,33,67,0.0,0.0,1.74,1.84,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,3.82,3.50,2.03,1.28,1.80,3.10,6.20,1.67,2.10,Gtech Community Stadium (Brentford- Middlesex),premier league,2022-08-13,16:30:00,10,4,0,4,30,13,17,21,1,22/23,55,57
15,NaN,Wolverhampton Wanderers,Fulham,John Brooks,2,0.0,0.0,1.58,1.21,0,0,0,0,0,0,5,3,3,0,3,0,2,1,1,2,7,10,2,0,5,10,7,14,60,40,0.0,0.0,0.87,0.93,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.96,3.26,3.26,1.36,2.02,3.63,7.25,1.80,1.95,Molineux Stadium (Wolverhampton- West Midlands),premier league,2022-08-13,14:00:00,8,6,0,6,17,2,15,21,0,22/23,31,54
14,NaN,Southampton,Leeds United,Tony Harrington,2,0.0,0.0,0.58,0.47,2,2,4,0,0,0,2,5,1,0,3,0,0,1,2,1,16,12,5,6,11,6,10,12,42,58,0.0,0.0,1.65,1.49,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.11,3.16,2.97,1.23,1.84,2.93,4.80,1.62,2.20,St. Mary's Stadium (Southampton- Hampshire),premier league,2022-08-13,14:00:00,7,4,0,4,28,11,17,22,0,22/23,35,47
11,60033.0,Arsenal,Leicester City,Darren England,2,0.0,0.0,2.37,0.79,4,2,6,2,2,0,6,2,1,0,1,0,0,1,1,0,20,8,8,4,12,4,15,9,50,50,0.0,0.0,2.29,0.99,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.68,3.50,4.07,1.23,1.86,2.79,5.20,1.80,1.95,Emirates Stadium (London),premier league,2022-08-13,14:00:00,8,2,0,2,28,12,16,24,1,22/23,88,51
12,31552.0,Brighton & Hove Albion,Newcastle United,Graham Scott,2,0.0,0.0,1.79,1.68,0,0,0,0,0,0,7,3,2,0,3,0,1,1,1,2,14,6,6,2,8,4,8,6,55,45,0.0,0.0,1.78,0.81,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.38,3.40,3.00,1.38,2.10,3.95,8.20,1.91,1.91,The American Express Community Stadium (Falmer...,premier league,2022-08-13,14:00:00,10,5,0,5,20,8,12,14,0,22/23,71,67
10,NaN,Aston Villa,Everton,Michael Oliver,2,0.0,0.0,2.00,0.79,2,1,3,1,1,0,5,8,4,0,1,0,1,3,1,0,13,16,5,5,8,11,9,7,58,42,0.0,0.0,1.64,1.59,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.91,3.26,3.40,1.34,1.95,3.60,7.40,1.80,1.95,Villa Park (Birmingham),premier league,2022-08-13,11:30:00,13,5,0,5,29,10,19,16,1,22/23,50,33
13,NaN,Manchester City,AFC Bournemouth,David Coote,2,0.0,0.0,2.74,0.89,4,0,4,3,3,0,11,1,0,0,3,0,0,0,1,2,21,4,9,2,12,2,9,9,67,33,0.0,0.0,2.53,0.48,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.12,5.99,22.12,1.11,1.36,2.05,3.20,3.00,1.36,Etihad Stadium (Manchester),premier league,2022-08-13,14:00:00,12,3,0,3,25,11,14,18,1,22/23,94,37
17,29281.0,Nottingham Forest,West Ham United,Robert Jones,2,0.0,0.0,1.58,0.63,1,0,1,1,1,0,6,7,4,0,1,0,0,4,1,0,14,20,7,6,7,14,10,11,43,57,0.0,0.0,1.72,2.09,0.0,0,0,0,0,0,0

In [21]:
df

,team_name,common_name,season,country,average_attendance_overall,average_attendance_home,average_attendance_away,minutes_per_goal_scored_overall,minutes_per_goal_scored_home,minutes_per_goal_scored_away,minutes_per_goal_conceded_overall,minutes_per_goal_conceded_home,minutes_per_goal_conceded_away,highest_goals_scored_home,highest_goals_scored_away,highest_goals_conceded_home,highest_goals_conceded_away,half_time_points_overall,half_time_points_home,half_time_points_away,corners_recorded_matches_num_overall,corners_recorded_matches_num_home,cornersRecorded_matches_away,corners_total_per_match_overall,corners_total_per_match_home,corners_total_per_match_away,corners_against_overall,corners_against_home,corners_against_away,corners_against_per_match_overall,corners_against_per_match_home,corners_against_per_match_away,over65_corners_num_overall,over75_corners_num_overall,over85_corners_num_overall,over95_corners_num_overall,over105_corners_num_overall,over115_corners_num_overall,over125_corners_num_overall,over135_corners_num_overall,over145_corners_num_overall,over65_corners_num_home,over75_corners_num_home,over85_corners_num_home,over95_corners_num_home,over105_corners_num_home,over115_corners_num_home,over125_corners_num_home,over135_corners_num_home,over145_corners_num_home,over65_corners_percentage_home,over75_corners_percentage_home,over85_corners_percentage_home,over95_corners_percentage_home,over105_corners_percentage_home,over115_corners_percentage_home,over125_corners_percentage_home,over135_corners_percentage_home,over145_corners_percentage_home,over65_corners_num_away,over75_corners_num_away,over85_corners_num_away,over95_corners_num_away,over105_corners_num_away,over115_corners_num_away,over125_corners_num_away,over135_corners_num_away,over145_corners_num_away,over65_corners_percentage_away,over75_corners_percentage_away,over85_corners_percentage_away,over95_corners_percentage_away,over105_corners_percentage_away,over115_corners_percentage_away,over125_corners_percentage_away,over135_corners_percentage_away,over145_corners_percentage_away,over25_corners_for_overall,over35_corners_for_overall,over45_corners_for_overall,over55_corners_for_overall,over65_corners_for_overall,over75_corners_for_overall,over85_corners_for_overall,over25_corners_for_percentage_overall,over35_corners_for_percentage_overall,over45_corners_for_percentage_overall,over55_corners_for_percentage_overall,over65_corners_for_percentage_overall,over75_corners_for_percentage_overall,over85_corners_for_percentage_overall,over25_corners_for_home,over35_corners_for_home,over45_corners_for_home,over55_corners_for_home,over65_corners_for_home,over75_corners_for_home,over85_corners_for_home,over25_corners_for_percentage_home,over35_corners_for_percentage_home,over45_corners_for_percentage_home,over55_corners_for_percentage_home,over65_corners_for_percentage_home,over75_corners_for_percentage_home,over85_corners_for_percentage_home,over25_corners_for_away,over35_corners_for_away,over45_corners_for_away,over55_corners_for_away,over65_corners_for_away,over75_corners_for_away,over85_corners_for_away,over25_corners_for_percentage_away,over35_corners_for_percentage_away,over45_corners_for_percentage_away,over55_corners_for_percentage_away,over65_corners_for_percentage_away,over75_corners_for_percentage_away,over85_corners_for_percentage_away,over25_corners_against_overall,over35_corners_against_overall,over45_corners_against_overall,over55_corners_against_overall,over65_corners_against_overall,over75_corners_against_overall,over85_corners_against_overall,over25_corners_against_percentage_overall,over35_corners_against_percentage_overall,over45_corners_against_percentage_overall,over55_corners_against_percentage_overall,over65_corners_against_percentage_overall,over75_corners_against_percentage_overall,over85_corners_against_percentage_overall,over25_corners_against_home,over35_corners_against_home,over45_corners_against_home,over55_corners_against_home,over65_cor

In [48]:
# Wyświetlenie rekordów, gdzie drużyna domowa lub gościnna to Liverpool
data[(data['home_team_name'] == "Liverpool") | (data['away_team_name'] == "Liverpool")]

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals,cumulative_away_goals
1,NaN,Fulham,Liverpool,Andy Madley,1,0.00,0.00,1.53,1.21,2,2,4,1,1,0,4,4,2,0,0,0,1,1,0,0,10,12,4,4,6,8,7,9,33,67,0.00,0.00,1.23,1.50,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,9.05,5.40,1.34,1.15,1.66,2.43,4.10,1.91,1.91,Craven Cottage (London),premier league,2022-08-06,11:30:00,8,2,0,2,22,8,14,16,0,22/23,52.0,68.0
19,NaN,Liverpool,Crystal Palace,Paul Tierney,2,0.00,0.00,2.32,0.89,1,1,2,1,0,1,4,2,2,1,4,0,1,2,0,4,25,8,5,4,20,4,7,8,73,27,0.00,0.00,2.56,0.92,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,1.22,6.50,15.00,1.17,1.60,2.30,4.30,2.20,1.62,Anfield (Liverpool),premier league,2022-08-15,19:00:00,6,6,1,7,33,9,24,15,0,22/23,68.0,38.0
29,NaN,Manchester United,Liverpool,Michael Oliver,3,0.00,1.00,2.53,1.21,2,1,3,1,1,0,6,8,3,0,1,0,2,1,1,0,13,18,5,6,8,12,11,7,30,70,2.01,1.50,1.43,2.16,3.50,100,100,100,50,0,50,100,50,100,10.00,4.00,5.23,4.46,1.64,1.17,1.50,2.30,4.00,1.57,2.25,Old Trafford (Manchester),premier league,2022-08-22,19:00:00,14,4,0,4,31,11,20,18,1,22/23,57.0,68.0
34,NaN,Liverpool,AFC Bournemouth,Stuart Attwell,4,1.00,0.00,2.32,0.89,9,0,9,5,5,0,8,1,0,0,1,0,0,0,1,0,21,6,14,3,7,3,6,5,69,31,2.56,0.48,2.83,0.70,3.00,50,100,50,50,0,50,100,0,100,5.00,7.00,1.11,9.40,23.00,1.11,1.35,2.00,3.15,2.50,1.50,Anfield (Liverpool),premier league,2022-08-27,14:00:00,9,1,0,1,27,17,10,11,1,22/23,68.0,37.0
48,NaN,Liverpool,Newcastle United,Andre Marriner,5,2.00,1.00,2.32,1.68,2,1,3,1,0,1,13,0,0,0,2,0,0,0,0,2,24,6,7,3,17,3,9,10,72,28,2.69,1.66,2.71,0.75,3.25,50,75,25,25,25,25,75,25,75,14.00,4.00,1.28,6.20,10.00,1.18,1.55,2.40,4.35,1.80,1.95,Anfield (Liverpool),premier league,2022-08-31,19:00:00,13,2,0,2,30,10,20,19,1,22/23,68.0,65.0
50,39240.0,Everton,Liverpool,Anthony Taylor,6,0.50,0.50,1.11,1.21,0,0,0,0,0,0,7,9,2,0,2,0,1,1,0,2,15,23,4,9,11,14,7,11,39,61,1.79,1.83,1.49,2.54,2.50,75,75,50,25,0,0,75,75,75,11.00,3.50,8.20,5.20,1.37,1.15,1.55,2.24,3.75,1.73,2.00,Goodison Park (Liverpool),premier league,2022-09-03,11:30:00,16,4,0,4,38,13,25,18,0,22/23,28.0,68.0
71,NaN,Liverpool,Brighton & Hove Albion,Andy Madley,9,2.33,2.00,2.32,1.47,3,3,6,3,1,2,9,2,1,0,1,0,0,1,1,0,21,7,10,7,11,0,9,13,54,46,2.70,1.40,2.50,1.25,3.67,67,100,67,17,17,33,84,50,100,11.33,2.00,1.33,4.80,9.50,1.20,1.60,2.58,4.70,1.91,1.91,Anfield (Liverpool),premier league,2022-10-01,14:00:00,11,2,0,2,28,17,11,22,0,22/23,68.0,70.0
84,NaN,Arsenal,Liverpool,Michael Oliver,10,3.00,0.67,2.37,1.21,3,2,5,3,2,1,4,3,1,0,2,0,1,0,1,1,12,9,8,4,4,5,7,8,44,56,2.59,2.07,1.68,1.13,3.17,84,84,84,42,13,25,71,84,84,14.50,2.